In [1]:
! pip install beir==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

/usr/local/lib/python3.11/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [20]:
# Download and load dataset
dataset = "trec-covid" # dataset name
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
out_dir = "datasets"
data_path = util.download_and_unzip(url, out_dir)

datasets/trec-covid.zip:   0%|          | 0.00/70.5M [00:00<?, ?iB/s]

In [21]:
# Load corpus, queries, and qrels
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/171332 [00:00<?, ?it/s]

In [22]:
import json
# load paraphrased dataset
with open(f"{dataset}_query_paraphrased_gpt4o.json", encoding='utf-8') as f:
    # Load the JSON data into a Python dictionary
    queries_para = json.load(f)

In [23]:
for q in queries_para:
  queries[q] = queries_para[q]['query_p']

In [24]:
import numpy as np
from collections import defaultdict
import string

def preprocess(text):
    # Basic normalization: lowercasing, punctuation removal, tokenization
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return text.split()

def compute_stats(corpus, queries, qrels):
    # Number of documents
    num_docs = len(corpus)

    # Number of queries (use only those present in qrels)
    num_queries = len(set(qrels.keys()))

    # Average document length (in words)
    doc_lengths = [len(preprocess(doc_data['text'])) for doc_data in corpus.values()]
    avg_doc_length = np.mean(doc_lengths)

    # Average query length (in words)
    query_lengths = [len(preprocess(queries[qid])) for qid in qrels if qid in queries]
    avg_query_length = np.mean(query_lengths)

    # Number of unique words (in corpus only)
    vocab = set()
    for doc_data in corpus.values():
        vocab.update(preprocess(doc_data['text']))
    num_unique_words = len(vocab)

    return {
        "num_docs": num_docs,
        "num_queries": num_queries,
        "avg_doc_length": avg_doc_length,
        "avg_query_length": avg_query_length,
        "num_unique_words": num_unique_words
    }


In [13]:
print("Dataset: ", dataset)
print('-' * 50)
stats = compute_stats(corpus, queries, qrels)
print(stats)

Dataset:  nfcorpus
--------------------------------------------------
{'num_docs': 3633, 'num_queries': 323, 'avg_doc_length': np.float64(219.6559317368566), 'avg_query_length': np.float64(5.343653250773994), 'num_unique_words': 36735}


In [19]:
print("Dataset: ", dataset)
print('-' * 50)
stats = compute_stats(corpus, queries, qrels)
print(stats)

Dataset:  scifact
--------------------------------------------------
{'num_docs': 5183, 'num_queries': 300, 'avg_doc_length': np.float64(200.8105344395138), 'avg_query_length': np.float64(14.63), 'num_unique_words': 50571}


In [25]:
print("Dataset: ", dataset)
print('-' * 50)
stats = compute_stats(corpus, queries, qrels)
print(stats)

Dataset:  trec-covid
--------------------------------------------------
{'num_docs': 171332, 'num_queries': 50, 'avg_doc_length': np.float64(148.19161044054817), 'avg_query_length': np.float64(11.86), 'num_unique_words': 383990}
